## 核心问题

1. fancydsp与客户点击GAP
2. fancydsp与秒针点击GAP

### 一、点击监测的业务知识点

1、媒体自身的点击监测，一般都是异步监测（具体要跟媒体确认）；

2、fancydsp的点击监测（监测系统数据），被腾讯放在异步，被优酷、爱奇艺、趣头条放在同步；

3、FTX的点击监测，一般都是异步监测；

4、fancy的三方监测，一般是异步的；

5、客户的三方监测，以客户给到的为准

    （1）在Amo的录入系统中，放在落地页一列，代码中包含监测代码，且能够打开的均为同步监测；
    （2）在Amo的录入系统中，放在点击监测1中，代码中包含监测代码，且能够打开的均为异步监测。

##### 点击支持同步流程

![点击监测上报流程](../imgs/点击监测上报流程.png)

### 二、常见问题

1、如果客户的点击监测为异步的，那么理论上跟ftx的数据是一致的，因为ftx一般都是异步监测；

2、如果客户的点击监测为同步的，它是通过fancy_dsp 302跳转到客户监测，其中会存在一部分正常损耗；

3、如果打开了屏蔽瞬时点击开关，规则是5s内点击2次，实际只会跳转一次；30s内如果在第1s、4s、18s各点击一次，实际也只会跳转2次（因为每隔5s内的重复点击只会跳转1次）；

4、iqiyi存在部分异常ua，目前排查过程中发现影响较大的异常ua有：IqiyiApp/iqiyi IqiyiVersion/10.6.0 、 IqiyiApp/iqiyi IqiyiVersion/10.6.5、IqiyiApp/iqiyi IqiyiVersion/10.6.6（其中10.6.0版本应该是6.12发布的，10.6.5版本是7.1号发布，10.6.6版本是7.4号发布），异常特征是：同一个request_id同步监测会上报多次，正常ua同步监测可以302跳转，第三方监测也可以上报，异常ua无法跳转（不支持302），三方监测不会上报；

5、如果实际排查过程中发现主要是爱奇艺gap，并且打开了屏蔽瞬时点击功能，则要同时考虑以下三种情况对客户（如果客户是同步监测）收数造成的影响：

    （1）打开屏蔽瞬时点击功能对客户收数的影响（block_jump=0）；
    （2）iqiyi的异常ua对客户收数的影响（user_agent not like 'IqiyiApp/iqiyi IqiyiVersion/%'）；
    （3）综合以上两点对收数的影响（block_jump=0 and user_agent not like 'IqiyiApp/iqiyi IqiyiVersion/%'）

6、根据历史排查经验来看，秒针点击数据是不去重的（不考虑重复点击），从以下两点可以得到验证：

    (1)ftx_total_click ≈ fancy_mz_click
    (2)ftx_有效_click < fancy_mz_click

7、关于IP库同步的问题，请求、曝光、点击三个可能不是同步更新的，对于同一个项目可能存在前一天曝光更新了IP，点击没有更新的情况；

8、国双监测，如果点击 > 曝光 会被判断为异常，因此运营通常会打开 屏蔽瞬时点击功能 ，其余监测没有这样的判断规则，因为打开此功能会导致双方数据差异，这点需要注意一下。

9、科大讯飞这个ADX的媒体很多，在运营使用科大讯飞进行通投时常出现fancydsp与秒针点击GAP问题，大部分情况为部分媒体或广告位有异常。此处可根据FTX与DSP(FTX≈秒针)的点击按广告位维度去对比，让运营将差别较大的广告位屏蔽掉。

In [4]:
# 准备工作
import numpy  as np
import pandas as pd
#连接presto库
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

# 本地则使用代理，默认socks5端口号为18888
import os
if os.environ.get('JUPYTERENV') != 'release':
    import socks
    import socket
    socks.set_default_proxy(socks.SOCKS5, "localhost", 18888)
    socket.socket = socks.socksocket

presto_engine = create_engine('presto://192.168.28.111:10080/hive/dwd')

运营提供数据信息
![点击数据对比](../imgs/clkgap/clkgap.png)
根据表中数据，我方DSP数据与我方秒针数据无GAP，点击存在GAP。

点击GAP可能的原因有：
1. 被反作弊过滤
2. 同步点击与异步点击收数不一致
3. 地域漂移

In [8]:
#查询我方有效点击情况
#广告组ID：250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250352,250353,250355,250356,250369,250370
#投放周期：2021-02-25至2021-03-04
#无地域定向
clkgap_df = pd.read_sql('''
select if(b.name like '%%粉色版%%','粉色版','钻石版') as "版本",count(if(ignore_rule=0,1,null)) as "有效点击",count(1) as "总点击"
from dwd.d_ad_clicklog_day a
join u6rds.ad_fancy.ad_group b on a.ext_adgroup_id = b.id
where thisdate = '2021-02-25'
  and ext_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250352,250353,250355,250356,250369,250370)
group by 1
''', presto_engine)
clkgap_df

,版本,有效点击,总点击
0,钻石版,8839,9834
1,粉色版,8720,9653


In [10]:
#查询我方异步点击情况
clk_df = pd.read_sql('''
select if(b.name like '%%粉色版%%','粉色版','钻石版') as "版本",count(ext_request_id) as "总点击"
from dwd.d_ad_ftx_click_data a
join u6rds.ad_fancy.ad_group b on a.ext_dsp_adgroup_id = b.id
where thisdate = '2021-02-25'
  and ext_dsp_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250352,250353,250355,250356,250369,250370)
group by 1
''', presto_engine)
clk_df

,版本,总点击
0,粉色版,5937
1,钻石版,6051


根据上述数据结果显示：
1. 反作弊确实过滤掉部分点击，但与秒针数据仍然有GAP
2. 我们异步点击收数与同步点击相差较大(与运营沟通了解该订单存在内购效果点击，该部分点击不经过FTX系统，因此重新整理数据，排除掉内购广告组250352,250353,250369,250370)

重新整理数据如下
![同异步点击数据对比](../imgs/clkgap/clkgap2.png)
此时发现我方异步数据与秒针数据一致，我方同步数据与我方异步数据差别较大，因此下面从媒体角度分析数据相差原因

In [13]:
#查询我方异步点击情况
impvendor_df = pd.read_sql('''
select ext_vendor_id as "媒体",count(a.ext_request_id) as "同步点击",count(b.ext_request_id) as "异步点击"
from 
(select ext_request_id,ext_vendor_id
from dwd.d_ad_clicklog_day a
join u6rds.ad_fancy.ad_group b on a.ext_adgroup_id = b.id
where thisdate = '2021-02-25'
  and ext_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250355,250356)
group by 1,2)a
left join 
(select ext_request_id
from dwd.d_ad_ftx_click_data
where thisdate = '2021-02-25'
  and ext_dsp_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250355,250356)
group by 1)b on a.ext_request_id=b.ext_request_id
group by 1
''', presto_engine)
impvendor_df

,媒体,同步点击,异步点击
0,165,11098,7827
1,52,1420,1418
2,142,1595,1581
3,118,587,584


In [14]:
#查询我方异步点击情况
impslot_df = pd.read_sql('''
select ext_slot_id as "媒体",count(a.ext_request_id) as "同步点击",count(b.ext_request_id) as "异步点击"
from 
(select ext_request_id,ext_slot_id
from dwd.d_ad_clicklog_day a
join u6rds.ad_fancy.ad_group b on a.ext_adgroup_id = b.id
where thisdate = '2021-02-25'
  and ext_vendor_id=165
  and ext_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250355,250356)
group by 1,2)a
left join 
(select ext_request_id
from dwd.d_ad_ftx_click_data
where thisdate = '2021-02-25'
  and ext_vendor_id=165
  and ext_dsp_adgroup_id in (250332,250333,250334,250335,250336,250337,250338,250339,250340,250341,250342,250343,250344,250345,250346,250347,250348,250349,250350,250351,250355,250356)
group by 1)b on a.ext_request_id=b.ext_request_id
group by 1
''', presto_engine)
impslot_df

,媒体,同步点击,异步点击
0,5B8DBADAFF12670CBD91AD6FC3493C32,60,60
1,AEB49D77E73D66FED350014C3698608C,22,22
2,B5CB44951B9C314BB593BA2DE8CDAACA,1,1
3,657A95AD6BC22D5A860583DD69E7ADD6,317,316
4,96297BC69432031FD0825E3994AFA1CE,4897,2314
5,B1C9B3845B694D59DF3F5ED4F4A70A70,1,0
6,C5EC1107AE0D69A19DCEABDF1703C165,293,293
7,79E541C2E7FDF6A85420265D77F06C89,4105,3895
8,6B7920A178EA24794394B2C2080E6419,18,18
9,5A01A0A1891D4EF933C70045C202315A,6,6


从媒体角度看，ADX科大讯飞(165)导致了同步点击与异步点击的差异，再细分科大讯飞的广告位看，其中96297BC69432031FD0825E3994AFA1CE、79E541C2E7FDF6A85420265D77F06C89、5520353AFE878D483B5171CCBBFC0306、FB2776B8691E2F12B838B0DFC756F97B。这四个广告位导致较多差异，汇报运营将这几个广告位屏蔽掉后再继续观察数据情况。